In [1]:
import soundfile as sf
from pytube import YouTube
from pydub import AudioSegment
import noisereduce as nr
import librosa
import pandas as pd
from googleapiclient.discovery import build
import os
import json
import subprocess
import librosa
import os
## ../AppSettings.py import 
import sys
sys.path.append("..")
from AppSettings import AppSetting

app_settings = AppSetting()

api_key = app_settings.get_secret()["GOOGLE"]["API_KEY"]


/home/aagunu/miniconda3/envs/DataCollectorEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/mnt/d/GITHUB/AI-Cover-Classification/Data Collector


In [2]:
def convert_to_wav(input_file_path):
    """
    Verilen ses dosyasını FFmpeg kullanarak WAV formatına dönüştürür.
    """
    # Çıktı dosyasının adını oluştur
    output_file_path = os.path.splitext(input_file_path)[0] + '.wav'
    
    # Dosya zaten WAV formatında mı kontrol et
    if input_file_path.endswith('.wav'):
        print(f"'{input_file_path}' already in WAV format.")
        return input_file_path
    
    # FFmpeg komutunu oluştur
    command = ['ffmpeg', '-y', '-i', input_file_path, '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '2', output_file_path]

    # FFmpeg komutunu çalıştır
    try:
        ##print (command)
        print(command)
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"Conversion successful: '{input_file_path}' to '{output_file_path}'")
        return output_file_path
    except subprocess.CalledProcessError as e:
        print(f"Error during conversion: {e}")
        return None

In [3]:
def video_indir_ve_wav_donustur(url, hedef_klasor='indirilenler'):
    if not os.path.exists(hedef_klasor):
        os.makedirs(hedef_klasor)
    try:
        video = YouTube(url)
        ses_dosyasi = video.streams.filter(only_audio=True).first()
        indirme_yolu = ses_dosyasi.download(output_path=hedef_klasor)
        yeni_dosya_adi = convert_to_wav(indirme_yolu)
        os.remove(indirme_yolu)
        return yeni_dosya_adi
    except Exception as e:
        print(f"Video indirme ve dönüştürme sırasında hata oluştu: {e}")
        return None

In [4]:
def sesi_isle_ve_bol(dosya_adi, sinif, index, sr=16000):
    y, sr = librosa.load(dosya_adi, sr=sr)
    y_reduced_noise = nr.reduce_noise(y=y, sr=sr)

    # İşlenmiş sesi geçici bir dosyaya kaydet
    temp_dosya = f'temp_{index}.wav'
    sf.write(temp_dosya, y_reduced_noise, sr)  # soundfile ile kaydet

    # 30 saniyelik segmentlere böl
    ses = AudioSegment.from_wav(temp_dosya)
    parca_uzunlugu = 30 * 1000
    for i in range(0, len(ses), parca_uzunlugu):
        parca = ses[i:i + parca_uzunlugu]
        parca_adi = f"{sinif}/{sinif}_{index}_{i//parca_uzunlugu}.wav"
        if not os.path.exists(sinif):
            os.makedirs(sinif)
        parca.export(parca_adi, format="wav")

    os.remove(temp_dosya)  # Geçici dosyayı sil

In [5]:


youtube = build('youtube', 'v3', developerKey=api_key)
def tr_char_to_non_tr_char(text):
    tr = "çğıöşüÇĞİÖŞÜ"
    non_tr = "cgiosuCGIOSU"
    for i in range(len(tr)):
        text = text.replace(tr[i], non_tr[i])
    ## Remove special characters
    text = ''.join(e for e in text if e.isalnum() or e == ' ')

    return text
def search_youtube(query):
    request = youtube.search().list(
        q=query,
        part="id,snippet",
        type="video",
        maxResults=200
    )
    response = request.execute()
    for item in response['items']:
        video_id = item['id']['videoId']
        video_title = tr_char_to_non_tr_char(item['snippet']['title'])
        video_link = f"https://www.youtube.com/watch?v={video_id}"
        print(f"Title: {video_title}\nLink: {video_link}\n")
        wav_dosyasi = video_indir_ve_wav_donustur(video_link)
        if wav_dosyasi is not None:
            sesi_isle_ve_bol(wav_dosyasi, "AI COVER", video_title)


In [6]:
search_youtube("AI COVER TÜRKÇE")

Title: Baris Manco Cem Karaca  Caw Bella   Ai Cover aicover  ai cover  aicovers
Link: https://www.youtube.com/watch?v=o_MgxcZ9VxM

['ffmpeg', '-y', '-i', '/mnt/d/GITHUB/AI-Cover-Classification/Data Collector/indirilenler/Barış Manço- Cem Karaca [ Çaw Bella ] - (Ai Cover) aicover  ai cover  aicovers.mp4', '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '2', '/mnt/d/GITHUB/AI-Cover-Classification/Data Collector/indirilenler/Barış Manço- Cem Karaca [ Çaw Bella ] - (Ai Cover) aicover  ai cover  aicovers.wav']
Conversion successful: '/mnt/d/GITHUB/AI-Cover-Classification/Data Collector/indirilenler/Barış Manço- Cem Karaca [ Çaw Bella ] - (Ai Cover) aicover  ai cover  aicovers.mp4' to '/mnt/d/GITHUB/AI-Cover-Classification/Data Collector/indirilenler/Barış Manço- Cem Karaca [ Çaw Bella ] - (Ai Cover) aicover  ai cover  aicovers.wav'
